In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
features = pd.read_pickle('./data/features_test.pkl')
stock_data = pd.read_csv('./data/stock_data_test.csv', index_col='Unnamed: 0')

In [3]:
display(features)
display(stock_data)

{'4584.T':           feat_1      feat_2     feat_3      feat_4      feat_5      feat_6  \
 180   759.772217  124.378746  35.910980 -355.077606  450.786469 -466.541962   
 181   780.926331  130.576157  35.047318 -363.625000  462.847321 -479.167175   
 182   785.210571  130.537582  36.907253 -366.455231  465.384583 -481.690155   
 183   724.858215  121.349426  31.804905 -340.099091  427.807434 -443.279755   
 184   715.975830  118.337715  32.622684 -333.855835  420.758728 -437.826019   
 ...          ...         ...        ...         ...         ...         ...   
 1248  240.497849   41.462242  13.131885 -109.656715  145.300552 -148.154724   
 1249  237.722244   43.641411  10.955653 -108.320877  143.320831 -149.069626   
 1250  244.596085   42.466618  12.683497 -111.768013  145.887741 -148.545349   
 1251  240.614731   40.725067  10.429011 -111.842094  145.893997 -151.098907   
 1252  238.514633   42.008884  13.450680 -108.465218  145.272293 -147.449829   
 
           feat_7      feat_

,4584.T,1557.T,8789.T,1893.T,MSFT
180,816.0,28420.0,149.0,537.0423,101.410780
181,822.0,28830.0,147.0,549.8493,102.930210
182,758.0,29120.0,137.0,547.2879,100.990320
183,749.0,28960.0,138.0,540.4575,101.974594
184,808.0,28870.0,138.0,532.7732,101.487230
...,...,...,...,...,...
1248,249.0,55370.0,65.0,631.0000,288.800000
1249,254.0,55640.0,67.0,635.0000,288.370000
1250,253.0,55630.0,67.0,629.0000,288.450000
1251,251.0,55640.0,67.0,628.0000,286.110000


In [4]:
from ray.tune import register_env

# 環境クラスのインポート
from gym_stock_trading_env import StockTradingEnv

# Define a function to create the environment
def create_stock_trading_env(_):
    env = StockTradingEnv(stock_data, features)
    return env

# Register the environment
env_name = "stock_trading_env"
register_env(env_name, create_stock_trading_env)

c:\Users\rodin\work\stock_trade\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# import gymnasium as gym
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.dqn import DQNConfig
from collections import OrderedDict

# PPOConfig.environment(disable_env_checking=True)
# DQNConfig.environment(disable_env_checking=True)

# Initialize Ray in local mode
ray.init(ignore_reinit_error=True, local_mode=True)

# Create the environment
env = StockTradingEnv(stock_data, features)

# Create PPO agent
ppo_config = PPOConfig().environment(env=env_name)
ppo_agent = ppo_config.build()

# Train the agents
num_iterations = 3

for i in range(num_iterations):
    ppo_result = ppo_agent.train()
    # dqn_result = dqn_agent.train()
    print(f"Iteration {i + 1}:")
    print(f"PPO: episode_reward_mean={ppo_result['episode_reward_mean']}")

# Evaluate the trained agents
def evaluate(agent, env):
    episode_reward = 0
    terminated = False
    obs = env.reset()[0]

    while not terminated:
        action = agent.compute_single_action(obs)
        obs, reward, terminated, _, _ = env.step(action)
        # Again convert OrderedDict to regular dict if necessary
        if isinstance(obs, OrderedDict):
            obs = dict(obs)
        episode_reward += reward

    return episode_reward

print("\nEvaluating trained agents:")
print(f"PPO: {evaluate(ppo_agent, env)}")

# Shutdown Ray
ray.shutdown()

2023-05-14 22:05:46,246	INFO worker.py:1625 -- Started a local Ray instance.
2023-05-14 22:05:47,728	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
:job_id:01000000
2023-05-14 22:05:47,766	WARNING worker.py:918 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
:actor_name:RolloutWorker
2023-05-14 22:05:47,772	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
:actor_name:RolloutWorker
2023-05-14 22:05:47,846	WARNING util.py:67 -- Install gputil for GPU system monitoring.


:job_id:01000000
:actor_name:RolloutWorker
:actor_name:RolloutWorker
Iteration 1:
PPO: episode_reward_mean=-68591025.8486963
Iteration 2:
PPO: episode_reward_mean=-8434367.945963096
Iteration 3:
PPO: episode_reward_mean=15601968.938144613

Evaluating trained agents:
PPO: -19868930.77381168
